In [1]:
addprocs(10);

In [10]:
@everywhere begin
    using NetworkDiscovery
    using POMDPs
    import POMDPs: solve
    using POMCP
    using POMDPToolbox
end

In [18]:
nodes = 100
comms = 5
probes = 10
p_inter = 0.01
p_intra = 0.3
N = 100

100

In [19]:
@everywhere solve(solver::DiscoveryHeuristic, pomdp) = solver
@everywhere function tick_hack(x::Float64,y::Float64)
           print(".")
           return x+y
end

In [20]:
function est_rew(solver, nodes, comms, probes, p_intra, p_inter, N)
    sum = @parallel (tick_hack) for i in 1:N
        prob_rng = MersenneTwister(i)
        sim_rng = MersenneTwister(i)
        nw = generate_network(prob_rng, nodes, comms, p_intra, p_inter)
        pomdp = generate_problem(prob_rng, nw, probes, 1, 100.0, 10, 10, p_intra, p_inter)
        policy = solve(solver, pomdp)
        revealed = initial_belief(pomdp)
        sim = RolloutSimulator(rng=sim_rng, initial_state=nw, initial_belief=revealed)
        simulate(sim, pomdp, policy)
    end
    # print("\r")
end

est_rew (generic function with 1 method)

In [21]:
guess_rng = MersenneTwister(1)
policy = DiscoveryHeuristic(ProbeHighestDegree(false), GuessBasedOnNeighbors(guess_rng))

DiscoveryHeuristic(ProbeHighestDegree(false),GuessBasedOnNeighbors(MersenneTwister(DSFMT_state(Int32[1749029653,1072851681,1610647787,1072862326,1841712345,1073426746,-198061126,1073322060,-156153802,1073567984  …  1977574422,1073209915,278919868,1072835605,1290372147,18858467,1815133874,-1716870370,382,0]),Uint32[0x00000001])))

In [22]:
@time est_rew(policy, nodes, comms, probes, p_intra, p_inter, N)

.........elapsed time: 0.095761203 seconds (579724 bytes allocated)


3800.0

In [25]:
rollout_rng = MersenneTwister(1)
pomcp_rng = MersenneTwister(1)
rollout_policy = DiscoveryHeuristic(ProbeHighestDegree(false), GuessBasedOnNeighbors(rollout_rng))
solver = POMCPSolver(rollout_policy, 0.0, 100.0, 1000, pomcp_rng, false, FullBeliefConverter(), 0)

POMCPSolver(DiscoveryHeuristic(ProbeHighestDegree(false),GuessBasedOnNeighbors(MersenneTwister(DSFMT_state(Int32[1749029653,1072851681,1610647787,1072862326,1841712345,1073426746,-198061126,1073322060,-156153802,1073567984  …  1977574422,1073209915,278919868,1072835605,1290372147,18858467,1815133874,-1716870370,382,0]),Uint32[0x00000001]))),0.0,100.0,1000,MersenneTwister(DSFMT_state(Int32[1749029653,1072851681,1610647787,1072862326,1841712345,1073426746,-198061126,1073322060,-156153802,1073567984  …  1977574422,1073209915,278919868,1072835605,1290372147,18858467,1815133874,-1716870370,382,-1]),Uint32[0x00000001]),false,FullBeliefConverter(),0)

In [26]:
@time est_rew(solver, nodes, comms, probes, p_intra, p_inter, N)

.........elapsed time: 627.653356452 seconds (649400 bytes allocated)


1400.0